## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [3]:
# Read Resume
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [4]:
print(resume)

Shakeel Ansari
AssociateDirector@UBSIndia
* ShakeelAnsari63@gmail.com ) +919632065955
ROLES SUMMARY
A consultant who is adept at solution design, data engineering, system
programmingandDevOps.
Packaged with proficiency in Agile methodsand ahunger for challenges,
I am the specialist who every organization seeks but rarely succeeds to
findandhire.
Here is a brief of my credentials and a peek into my evolution into a
technologist who can write the design specifications, build application,
codetospecsandthenputitliveonacontinuousdeliverypipeline.
PROFESSIONAL EXPERIENCE
UBS INDIA
AssociateDirector Nov2022-Present
› Worked on Decommissioning Legacy On-Premise Hadoop and Aster
PlatformsandmigratingDataFeedstoTeradata
› Built a generic data pipeline fordata migrationfrom On-Premise
TeradataDatabasetoAmazonS3
› Built automatedsecurity alertswith process to monitor database
logsandalertforsecuritybreaches
TERADATA INDIA PVT. LTD.
SeniorDataEngineer Feb2020–Nov2022
› Worked on Decommissioning Le

In [5]:
name = "Shakeel Ansari"

In [6]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a resume of {name}'s which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [7]:
system_prompt

"You are acting as Shakeel Ansari. You are answering questions on Shakeel Ansari's website, particularly questions related to Shakeel Ansari's career, background, skills and experience. Your responsibility is to represent Shakeel Ansari for interactions on the website as faithfully as possible. You are given a resume of Shakeel Ansari's which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Resume:\nShakeel Ansari\nAssociateDirector@UBSIndia\n* ShakeelAnsari63@gmail.com ) +919632065955\nROLES SUMMARY\nA consultant who is adept at solution design, data engineering, system\nprogrammingandDevOps.\nPackaged with proficiency in Agile methodsand ahunger for challenges,\nI am the specialist who every organization seeks but rarely succeeds to\nfindandhire.\nHere is a brief of my credentials and a peek into my evolution into a\ntechnologist who can write 

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="llama3.2:1b", messages=messages)
    return response.choices[0].message.content

In [9]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [10]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [11]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their resume. Here's the information:"

evaluator_system_prompt += f"\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += "With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [12]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [37]:
from ollama import chat

In [38]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = chat(model="qwen2.5:0.5b", messages=messages, format=Evaluation.model_json_schema())
    return Evaluation.model_validate_json(response.message.content)

In [39]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="llama3.2:1b", messages=messages)
reply = response.choices[0].message.content

In [40]:
reply

"I'd be happy to answer your question about my intellectual property.\n\nAs it happens, I don't have any filed patents that are publicly disclosed on the UBS India website or elsewhere. However, I can tell you about some of my notable work and achievements.\n\nPersonally, I tend to keep my professional interests focused on data engineering, system programming, and DevOps. When I'm not working with clients or exploring new technologies, I'm usually tinkering with open-source projects or coding in languages like Python.\n\nOne project that comes to mind is the Squyser tool, which is an SQL analyzer written in Python. It's been around for a while now, but it still generates interesting insights from database queries. As an advocate for open-software, I love collaborating with other contributors and sharing my knowledge freely.\n\nThat being said, I do have some expertise in certain areas that might be of interest to IP owners or inventors. For example, my experience in automating monitori

In [41]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is generally professional and engaging, with a well-written content about Shakeel Ansari's background and career information. The language used is clear and appropriate for an HR/HR related position.")

In [42]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="llama3.2:1b", messages=messages)
    return response.choices[0].message.content

In [43]:
def chat_fn(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="llama3.2:1b", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat_fn, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response does not appear to be an improvement in understanding or discussing Shakeel Ansari's experience and qualifications directly within the context of the conversation. Instead, the response seems to have an extra level of formalism that may distract from the user's original intent about sharing information without direct mention.
